In [2]:
import torch, math, os, numpy as np
L, Nt, beta = 16, 32, 2.2
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def random_u2():
    theta = torch.rand(1, device=device) * 2 * math.pi
    return torch.stack([torch.cos(theta), torch.sin(theta)], dim=-1)  # (1,2)

# --- DÜZELTİLMİŞ SATIR ---
U = torch.stack(
        [torch.stack([random_u2() for _ in range(4)], dim=0)
         for _ in range(L**3 * Nt)],
        dim=0
    )
# -------------------------

def staple_sum(U, idx):
    return sum(U[(idx + k) % U.shape[0]] for k in (1, 2, 3))

def sweep(U):
    accept = 0
    for idx in range(U.shape[0]):
        old = U[idx].clone()
        proposal = torch.stack([random_u2() for _ in range(4)], dim=0)
        dS = -beta * ((proposal - old) * staple_sum(U, idx)).sum()
        if dS < 0 or torch.rand(1, device=device) < torch.exp(-dS):
            U[idx] = proposal
            accept += 1
    return accept / U.shape[0]

for it in range(10):
    acc = sweep(U)
    print(f'Iter {it}: accept={acc:.2f}')

plaq = U.mean().item()
print("plaquette =", plaq)

os.makedirs('out', exist_ok=True)
np.savez('out/mini_lattice_results.npz', plaquette=plaq, beta=beta)


Iter 0: accept=0.55
Iter 1: accept=0.47
Iter 2: accept=0.43
Iter 3: accept=0.40
Iter 4: accept=0.39
Iter 5: accept=0.37
Iter 6: accept=0.37
Iter 7: accept=0.36
Iter 8: accept=0.35
Iter 9: accept=0.35
plaquette = -0.0002654680283740163
